### Dairo Alberto Cuervo Garcia
#### Proyecto de grado - Exploración de discurso en cartas de maximos responsables - Maestría en Ciencias de los Datos y Analítica - Universidad EAFIT - 2024/1

## Análisis de Sentimiento con TextBlob

In [1]:
# librerias necesarias
import os
#Cargar librerias necesarias para TextBlob
import pandas as pd
from textblob import TextBlob

# Cargar librerias necesarias para VADER
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DairoAlbertoCuervoGa\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# leer archivo resultados csv
current_dir = os.path.dirname(os.path.abspath('__file__'))
path_csv= os.path.join(current_dir, '1.1.Output', 'Resultados.csv')
df = pd.read_csv(path_csv)

#### SA con Textblob
TextBlob es una biblioteca de Python para procesar datos de texto. Proporciona una API para realizar tareas de
procesamiento de lenguaje natural (PLN), (análisis de sentimientos, tokenización, clasificación de texto, corrección ortográfica). Esta Basada en las librerías NLTK y Pattern.


In [3]:
# Función para analizar el sentimiento basado en la polaridad
def analizar_sentimiento(texto):
    blob = TextBlob(texto)
    polaridad = blob.sentiment.polarity
    if polaridad > 0.01:
        return 'Positivo', polaridad
    elif polaridad < -0.01:
        return 'Negativo', polaridad
    else:
        return 'Neutro', polaridad


# Aplicar análisis de sentimientos
df['Sentimiento_Textblob'], df['Polaridad_Textblob'] = zip(*df['texto_lematizado'].apply(analizar_sentimiento))

# Muestra los primeros registros para verificar
print(df[['Empresa_Año', 'Sentimiento_Textblob', 'Polaridad_Textblob']].head())

          Empresa_Año Sentimiento_Textblob  Polaridad_Textblob
0  BancodeBogota_2018             Positivo            0.041398
1  BancodeBogota_2019               Neutro            0.002857
2  BancodeBogota_2020             Positivo            0.039431
3  BancodeBogota_2021             Positivo            0.022727
4  BancodeBogota_2022             Positivo            0.087356


### Almacenar polaridad de Textblob en csv de Resultados

In [4]:
df.to_csv(path_csv, index=False) #almacenar en resultados.csv

## Análisis de Sentimiento con NLTK-Vader
La herramienta VADER  (Valence Aware Dictionary and sEntiment Reasoner) incluida en la biblioteca NLTK. es un modelo de análisis de sentimientos especialmente afinado para sentimientos expresados contextos informales, buscando la intensidad.

In [5]:
# Crear un analizador de sentimiento de VADER globalmente
sia = SentimentIntensityAnalyzer()

In [6]:
# Definir una función para aplicar el análisis de sentimiento y retornar la categoría y la polaridad
def analizar_sentimiento(texto):
    scores = sia.polarity_scores(texto)
    compound = scores['compound']
    if compound > 0.05:
        return ('Positivo', compound)
    elif compound < -0.05:
        return ('Negativo', compound)
    else:
        return ('Neutro', compound)

# Aplicar análisis de sentimientos
df['Sentimiento_VADER'], df['Polaridad_VADER'] = zip(*df['texto_lematizado'].apply(analizar_sentimiento))

# Muestra los primeros registros para verificar
print(df[['Empresa_Año', 'Sentimiento_VADER', 'Polaridad_VADER']].head())

          Empresa_Año Sentimiento_VADER  Polaridad_VADER
0  BancodeBogota_2018          Positivo           0.7650
1  BancodeBogota_2019          Positivo           0.9169
2  BancodeBogota_2020          Negativo          -0.8689
3  BancodeBogota_2021          Positivo           0.9805
4  BancodeBogota_2022          Positivo           0.9878


### Almacenar resultados de VADER en csv de Resultados

In [7]:
df.to_csv(path_csv, index=False) #almacenar resultados en Resultados.csv

## SA Sitema Híbrido
construcción de un modelo híbrido que combine ambos SA para no perder expresiones negativas, en este se priorizan la negatividad de los sentimientos dado que TextBlob mide la subjetividad del texto y VADER capta la polaridad basado en la intensidad del sentimiento expresado:

In [8]:
# Función para determinar sentimiento híbrido, priorizando la negatividad
def determinar_sentimiento_hibrido(row):
    if 'Negativo' in [row['Sentimiento_Textblob'], row['Sentimiento_VADER']]:
        return 'Negativo'
    elif 'Neutro' in [row['Sentimiento_Textblob'], row['Sentimiento_VADER']]:
        if 'Positivo' in [row['Sentimiento_Textblob'], row['Sentimiento_VADER']]:
            return 'Neutro'
        return 'Neutro'
    else:
        return 'Positivo'

# Aplicar la función a cada fila del DataFrame
df['Sentimiento_Sistema_Hibrido'] = df.apply(determinar_sentimiento_hibrido, axis=1)
# Muestra los primeros registros para verificar
print(df[['Sentimiento_Textblob', 'Sentimiento_VADER', 'Sentimiento_Sistema_Hibrido']].head())

  Sentimiento_Textblob Sentimiento_VADER Sentimiento_Sistema_Hibrido
0             Positivo          Positivo                    Positivo
1               Neutro          Positivo                      Neutro
2             Positivo          Negativo                    Negativo
3             Positivo          Positivo                    Positivo
4             Positivo          Positivo                    Positivo


### Almacenar resultados de Sistema Híbrido en csv de Resultados

In [9]:
df.to_csv(path_csv, index=False) #almacenar en Resultados.csv